In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0
%reload_ext autoreload

In [0]:
!pip install -U imbalanced-learn

In [0]:
import threadpoolctl
threadpoolctl.threadpool_info() 

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import preprocessing
from preprocessing import *
import warnings
warnings.filterwarnings("ignore")

In [0]:
data= pd.read_csv("emi_prediction_dataset.csv", low_memory=False)

In [0]:
data.isna().sum()


In [0]:
pre=quality_check(data, Prediction=False)

In [0]:
n_data = pre.final_processsing().emi_pred

In [0]:
n_data.shape

In [0]:
n_data.isna().sum()

In [0]:
n_data.shape

In [0]:
n_data

In [0]:
target_cols=['emi_eligibility', 'max_monthly_emi']

In [0]:
n_data.columns

In [0]:
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import balanced_accuracy_score, cohen_kappa_score, matthews_corrcoef, precision_recall_fscore_support, confusion_matrix, ConfusionMatrixDisplay, classification_report
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [0]:
def data_prepare(data, datatype="type1", target_type=None):
    final_columns_v1 = [
    'gender', 'marital_status', 'education', 'monthly_salary',
       'employment_type', 'years_of_employment', 'company_type', 'house_type',
       'monthly_rent', 'family_size', 'dependents', 'school_fees',
       'college_fees', 'travel_expenses', 'groceries_utilities',
       'other_monthly_expenses', 'existing_loans', 'current_emi_amount',
       'credit_score', 'bank_balance', 'emergency_fund', 'emi_scenario',
       'requested_amount', 'requested_tenure']
    final_columns_v2=[
     'gender', 'marital_status', 'education', 'monthly_salary',
       'employment_type', 'years_of_employment', 'company_type', 'house_type',
       'monthly_rent', 'family_size', 'dependents',  'existing_loans', 'credit_score', 'emi_scenario',
       'requested_amount', 'requested_tenure', 'Total_Fixed_expenses',
       'Total_Other_expenses','Savings']
    if datatype=="type1":
        if target_type==None:
            final_data=data[final_columns_v1]
        elif target_type=="regression":
            final_data=data[final_columns_v1]
            final_data['max_monthly_emi']=data["max_monthly_emi"]
        elif target_type=="classification":
            final_data=data[final_columns_v1]
            final_data['emi_eligibility']=data["emi_eligibility"]
        else:
            print("Please enter correct target type")
    elif datatype=="type2":
        if target_type==None:
            final_data=data[final_columns_v2]
        elif target_type=="regression":
            final_data=data[final_columns_v2]
            final_data['max_monthly_emi']=data["max_monthly_emi"]
        elif target_type=="classification":
            final_data=data[final_columns_v2]
            final_data['emi_eligibility']=data["emi_eligibility"]
    return final_data

In [0]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
def data_split(cdata, rebalance = False):
    X=cdata.drop("emi_eligibility", axis=1)
    y=cdata["emi_eligibility"]
    if rebalance==True:
        sm = SMOTE(random_state=42)
        X, y = sm.fit_resample(X, y)
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    return x_train, x_test, y_train, y_test

In [0]:
import mlflow
mlflow.set_registry_uri("databricks-uc")

In [0]:
%pip install optuna

In [0]:
import optuna

In [0]:
n_data

In [0]:
import mlflow

In [0]:
target_cols

In [0]:
try:    mlflow.create_experiment("/Workspace/Users/sushantkashikar1@gmail.com/mlflow/RandomForestRegressor_Optuna")
except Exception as e:
    if 'RESOURCE_ALREADY_EXISTS' in str(e):
        experiment_id = mlflow.get_experiment_by_name("/Workspace/Users/sushantkashikar1@gmail.com/mlflow/RandomForestRegressor_Optuna").experiment_id
    else:
        raise


In [0]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
import pandas as pd
import numpy as np
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score,
    mean_absolute_percentage_error,
    explained_variance_score,
    median_absolute_error
)

# Load dataset (replace with your actual EMI dataset)
final_data = data_prepare(n_data, datatype="type1", target_type="regression")
X=final_data.drop([target_cols[1]], axis=1)
y=final_data[target_cols[1]]

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Set up MLflow experiment
mlflow.set_experiment("/Workspace/Users/sushantkashikar1@gmail.com/mlflow/RandomForestRegressor_Optuna")

# Define objective function for Optuna
def objective(trial):
    with mlflow.start_run(nested=True):
        # Hyperparameters to tune
        n_estimators = trial.suggest_int("n_estimators", 300, 500)
        max_depth = trial.suggest_int("max_depth", 3, 20)
        min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
        min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 5)
        max_features = trial.suggest_categorical("max_features", ["sqrt", "log2", None])

        # Model
        model = RandomForestRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            max_features=max_features,
            random_state=42,
            n_jobs=-1
        )

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log parameters & metric to MLflow
        mlflow.log_params(trial.params)
        #mlflow.log_metric("cv_RMSE", rmse)
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        mae = mean_absolute_error(y_test, y_pred)
        mape = mean_absolute_percentage_error(y_test, y_pred) * 100
        r2 = r2_score(y_test, y_pred)
        evs = explained_variance_score(y_test, y_pred)
        medae = median_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        std_error = np.std(y_test - y_pred)

        # Log parameters and metrics for the trial
        
        mlflow.log_metrics({
            "RMSE": rmse,
            "MAE": mae,
            "MAPE(%)": mape,
            "R2": r2,
            "ExplainedVariance": evs,
            "MedianAE": medae,
            "MSE": mse,
            "Std_Error": std_error
        })

        return rmse

# Create and run Optuna study
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=10, n_jobs=1)

# Best hyperparameters
print("Best parameters:", study.best_params)

# Train final model with best parameters
best_params = study.best_params
best_model = RandomForestRegressor(**best_params, random_state=42, n_jobs=-1)
best_model.fit(X_train, y_train)

# Evaluate on test data
y_pred = best_model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Log final model
with mlflow.start_run(run_name="Best_RF_Model"):
    mlflow.log_params(best_params)
    mlflow.log_metric("RMSE", rmse)
    mlflow.log_metric("MAE", mae)
    mlflow.log_metric("R2", r2)
    sign=mlflow.models.infer_signature(X_train, best_model.predict(X_train))
    mlflow.sklearn.log_model(best_model, "model", signature=sign)

print(f"Test RMSE: {rmse:.2f}, MAE: {mae:.2f}, R2: {r2:.3f}")


In [0]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import pandas as pd
import numpy as np
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score,
    mean_absolute_percentage_error,
    explained_variance_score,
    median_absolute_error
)

with open("assets/v1_scaler.pkl", "rb") as f:
    v1_scaler = pickle.load(f)
with open("assets/v2_scaler.pkl", "rb") as f:
    v2_scaler = pickle.load(f)



# Load dataset (replace with your actual EMI dataset)
final_data = data_prepare(n_data, datatype="type1", target_type="regression")
X=final_data.drop([target_cols[1]], axis=1)
y=final_data[target_cols[1]]

X= pd.DataFrame(v1_scaler.transform(X), columns=X.columns)



# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Set up MLflow experiment
mlflow.set_experiment("/Workspace/Users/sushantkashikar1@gmail.com/mlflow/RandomForestRegressor_Optuna")

# Define objective function for Optuna
def objective(trial):
    with mlflow.start_run(nested=True):
        # Hyperparameters to tune
        n_estimators = trial.suggest_int("n_estimators", 300, 500)
        max_depth = trial.suggest_int("max_depth", 3, 20)
        min_samples_split = trial.suggest_int("min_samples_split", 4, 15)
        min_samples_leaf = trial.suggest_int("min_samples_leaf", 3, 15)
        max_features = trial.suggest_categorical("max_features", ["sqrt", "log2", None])

        # Model
        model = RandomForestRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            max_features=max_features,
            random_state=42,
            n_jobs=-1
        )

        # Cross-validation
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log parameters & metric to MLflow
        mlflow.log_params(trial.params)
        #mlflow.log_metric("cv_RMSE", rmse)
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        mae = mean_absolute_error(y_test, y_pred)
        mape = mean_absolute_percentage_error(y_test, y_pred) * 100
        r2 = r2_score(y_test, y_pred)
        evs = explained_variance_score(y_test, y_pred)
        medae = median_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        std_error = np.std(y_test - y_pred)

        # Log parameters and metrics for the trial
        
        mlflow.log_metrics({
            "RMSE": rmse,
            "MAE": mae,
            "MAPE(%)": mape,
            "R2": r2,
            "ExplainedVariance": evs,
            "MedianAE": medae,
            "MSE": mse,
            "Std_Error": std_error
        })

        return rmse

# Create and run Optuna study
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=10, n_jobs=1)

# Best hyperparameters
print("Best parameters:", study.best_params)

# Train final model with best parameters
best_params = study.best_params
best_model = RandomForestRegressor(**best_params, random_state=42, n_jobs=-1)
best_model.fit(X_train, y_train)

# Evaluate on test data
y_pred = best_model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Log final model
with mlflow.start_run(run_name="Best_RF_Model"):
    mlflow.log_params(best_params)
    mlflow.log_metric("RMSE", rmse)
    mlflow.log_metric("MAE", mae)
    mlflow.log_metric("R2", r2)
    sign=mlflow.models.infer_signature(X_train, best_model.predict(X_train))
    mlflow.sklearn.log_model(best_model, "model", signature=sign)

print(f"Test RMSE: {rmse:.2f}, MAE: {mae:.2f}, R2: {r2:.3f}")


In [0]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import pandas as pd
import numpy as np
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score,
    mean_absolute_percentage_error,
    explained_variance_score,
    median_absolute_error
)


with open("assets/v1_scaler.pkl", "rb") as f:
    v1_scaler = pickle.load(f)
with open("assets/v2_scaler.pkl", "rb") as f:
    v2_scaler = pickle.load(f)


n_data["Savings"] = n_data["bank_balance"] + n_data["emergency_fund"]

# Load dataset (replace with your actual EMI dataset)
final_data = data_prepare(n_data, datatype="type2", target_type="regression")
X=final_data.drop([target_cols[1]], axis=1)
y=final_data[target_cols[1]]

X= pd.DataFrame(v2_scaler.transform(X), columns=X.columns)



# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Set up MLflow experiment
mlflow.set_experiment("/Workspace/Users/sushantkashikar1@gmail.com/mlflow/RandomForestRegressor_Optuna")

# Define objective function for Optuna
def objective(trial):
    with mlflow.start_run(nested=True):
        # Hyperparameters to tune
        n_estimators = trial.suggest_int("n_estimators", 300, 500)
        max_depth = trial.suggest_int("max_depth", 3, 20)
        min_samples_split = trial.suggest_int("min_samples_split", 2, 15)
        min_samples_leaf = trial.suggest_int("min_samples_leaf", 3, 15)
        max_features = trial.suggest_categorical("max_features", ["sqrt", "log2", None])

        # Model
        model = RandomForestRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            max_features=max_features,
            random_state=42,
            n_jobs=-1
        )

        # Cross-validation
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log parameters & metric to MLflow
        mlflow.log_params(trial.params)
        
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        mae = mean_absolute_error(y_test, y_pred)
        mape = mean_absolute_percentage_error(y_test, y_pred) * 100
        r2 = r2_score(y_test, y_pred)
        evs = explained_variance_score(y_test, y_pred)
        medae = median_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        std_error = np.std(y_test - y_pred)

        # Log parameters and metrics for the trial
        
        mlflow.log_metrics({
            "RMSE": rmse,
            "MAE": mae,
            "MAPE(%)": mape,
            "R2": r2,
            "ExplainedVariance": evs,
            "MedianAE": medae,
            "MSE": mse,
            "Std_Error": std_error
        })

        return rmse

# Create and run Optuna study
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=10, n_jobs=1)

# Best hyperparameters
print("Best parameters:", study.best_params)

# Train final model with best parameters
best_params = study.best_params
best_model = RandomForestRegressor(**best_params, random_state=42, n_jobs=-1)
best_model.fit(X_train, y_train)

# Evaluate on test data
y_pred = best_model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Log final model
with mlflow.start_run(run_name="Best_RF_Model"):
    mlflow.log_params(best_params)
    mlflow.log_metric("RMSE", rmse)
    mlflow.log_metric("MAE", mae)
    mlflow.log_metric("R2", r2)
    sign=mlflow.models.infer_signature(X_train, best_model.predict(X_train))
    mlflow.sklearn.log_model(best_model, "model", signature=sign)

print(f"Test RMSE: {rmse:.2f}, MAE: {mae:.2f}, R2: {r2:.3f}")
